In [1]:
def dirichlet_noise(size, eta=None):
    if eta is None:
        eta = 8/(size)
#     this is only valid if the legal moves matches the size of the input array
#     eta = 8/size
#     ~10 legal moves seems like it would be about .9
# should work out the equation at some point
    noise_vec = []

    for _ in range(size):
        noise_vec.append(eta)

    noise = np.random.dirichlet(noise_vec, 1)
    return noise[0]

In [2]:
import numpy as np
from IPython.core.debugger import set_trace
# import random
# import bisect
# import collections

# def cdf(weights):
#     total = sum(weights)
#     result = []
#     cumsum = 0
#     for w in weights:
#         cumsum += w
#         result.append(cumsum / total)
#     return result

# def choice(population, weights):
#     assert len(population) == len(weights)
#     cdf_vals = cdf(weights)
#     x = random.random
#     idx = bisect.bisect(cdf_vals, x)
#     return population[idx]

# idea: instead of explicitly make tree, could use sqlite or something to keep track of stuff
# it might significantly slow down things since it would read from disc.
def create_node(parent, action, proba):
    print("in create node")
#     if proba is None and state is None:
#         node["true_root"] = True
#         node["depth"] = 0
    node = {}
    node["children"] = []
    node["visits"] = 0
    node["total_value"] = 0
    node["mean_value"] = 0
    node["probability"] = proba
    node["parent"] = parent
    node["action"] = action
    node["depth"] = parent["depth"] + 1
#     if state is not None:
#         node["parent"] = state
#         node["action"] = action
#     else:
#         node["depth"] = 0
#         node["is_root"] = True
    return node

def select(node, is_stochastic=False, most_visits=False, T=1):
    print("in select")
    scores = []
    probabilities = []
    c = np.log(3)
    all_visits = np.sum([child["visits"] for child in node["children"]])
    for i, child in enumerate(node["children"]):
#         visits = all_visits - child["visits"]
#         if node["depth"] > 0 and (node["depth"] % (3)) == 0:
#             set_trace()
#         U = c * child["probability"] * np.log(visits+1)/(1 + child["visits"])
        U = child["probability"]/(1 + child["visits"])
        if U == -np.inf:
            set_trace()
        print("U: {}, child visits: {}, child proba: {}, child mean value: {}".format(U, child["visits"],
                                                                                     child["probability"],
                                                                                     child["mean_value"]))
        score = child["mean_value"] + U
        scores.append(score)
        if all_visits == 0:
            proba = 0
        else:
            proba = (child["visits"]**(1/T))/all_visits
        probabilities.append(proba)
        
    probabilities = np.array(probabilities)
    if np.sum(probabilities) == 0:
        probabilities = np.zeros(len(probabilities))
        probabilities += 1/len(probabilities)
    
    if is_stochastic:
        node = np.random.choice(node["children"], p = probabilities)
    elif most_visits:
        node = node["children"][np.argmax(probabilities)]
    else:
#         print("end of select scores", scores)
#         print("end of select probas", probabilities)
        node = node["children"][np.argmax(scores)]
    
    set_trace()
    test = "hi"
    return node, probabilities
    
def _transition(state, node):
    print("in transition")
#     print("depth: {}".format(node["depth"]))
    state.board[node["depth"]] = node["action"]
    if state.is_root:
        state.is_root = False
            
def expand(state, node):
    print("in expand")
    ohe_board = state.lb.transform(state.board)
#     skips = []
#     non_skips = []
#     for i, ohe in enumerate(ohe_board):
#         if i % (state.max_width+1) == 0:
#             non_skips.append(ohe)
#         else:
#             skips.append(ohe)
#     non_skips = np.array([non_skips])
#     skips = np.array([skips])
    ohe_board = np.expand_dims(ohe_board, axis=0)
#     so I want to change the inputs to be OHE's of the skip connections sent to
# convs, and OHE's of 
#     (probabilities, value) = state.net.predict([non_skips, skips])
    (probabilities, value) = state.net.predict(ohe_board)
    probabilities = probabilities[0]
    value = value[0][0]
#     print("Probabilities and value post net: {}, {}".format(probabilities, value))
#     if state.is_root:
    probabilities += dirichlet_noise(size=len(probabilities))
    probabilities /= 2
    for i, proba in enumerate(probabilities):
        action = state.layer_types_numeric[i]
        n = create_node(node, action, proba)
        node["children"].append(n)
        
#     print(probabilities)
        
    return value
        
def backup(node, value, orig_depth):
    print("in backup")
    while "parent" in node:
#         print("value")
#         if value is not None:
#             print("passed value test")
        if node["depth"] > orig_depth:
            node["visits"] += 1
            node["total_value"] += value
    #         not sure about dividing, by visits, seems like it should be by total number of moves or something
            node["mean_value"] = node["total_value"]/node["visits"]
        node = node["parent"]
        
#     print(node["depth"])
        
    return node

In [3]:
def convert_to_string(arr):
    string_ver = ""
    for num in arr:
        string_ver += str(num) + "_"
    string_ver = string_ver[:-1]
    return string_ver

In [4]:
from copy import deepcopy
from sklearn.preprocessing import LabelBinarizer
from model import create_net
from tqdm import tqdm
from evaluate_arch import eval_arch

# def default_layer_types():
#     layer_types = ["conv2d1x1", "conv2d3x3", "conv2dsep1x1", "conv2dsep3x3"]
#     return layer_types

class arch_state():
    def __init__(self, kwargs={}):
        self.layer_types = kwargs.get("layer_types", 
                   ["", "conv2d1x1", "conv2d3x3",
#                     "conv2dsep1x1", "conv2dsep3x3",
                   "conv2d1x1stride", "conv2d3x3stride"
                   ])
        self.layer_types_numeric = kwargs.get("layer_types_numeric", 
                                              [i for i in range(len(self.layer_types))])
        self.max_width = kwargs.get("max_width", 2)
        self.max_depth = kwargs.get("max_depth", 10)
        size = 0
        for i in range(self.max_width):
            size += self.max_depth-(i+1)
        self.size = kwargs.get("size", size)
        self.board = kwargs.get("board", np.zeros(self.size, dtype=int))
        self.is_root = kwargs.get("is_root", True)
        self.net = kwargs.get("net", create_net(time_steps=self.size, vector_size=len(self.layer_types)))
        self.lb = kwargs.get("lb", LabelBinarizer())
        self.lb.fit(self.layer_types)
        
#     def compare_to_best(self, node)
#         result, model = eval_arch(self, node, vs_best=True)
        
    def evaluate(self, node):
        return eval_arch(self, node)
            
    def clone(self):
        kwargs = {}
        kwargs["layer_types"] = self.layer_types
        kwargs["layer_types_numeric"] = self.layer_types_numeric
        kwargs["max_width"] = self.max_width
        kwargs["max_depth"] = self.max_depth
        kwargs["size"] = self.size
        kwargs["board"] = self.board
        kwargs["is_root"] = self.is_root
        kwargs["net"] = self.net
        kwargs["lb"] = self.lb
        return arch_state(kwargs)

# def get_max_depth(board):

# a state must specify current position
# for now why dont we assume that it starts from the root

# for now going to assume that the state is the default state
# in the future, I could add so that you can start from an arbitrary state and look up the
# tree from there

def MCTS(root_state=None, root_node=None, is_stochastic=False, num_simulations=5, records=[]):
#     if state["curr_position"] == 0:
#         state = create_node()

    if root_state is None:
        root_state = arch_state()
        
#     eventually want to allow node to be able to be passed in
# maybe want to look into storing children as part of the state
    if root_node is None:
        node = {"children": [], "depth": -1}
        root_node = {"children": [], "depth": -1}
    else:
        node = deepcopy(root_node)
    
#     so, what do I want.
# I want to be able to input a raw board position / graph, and have that act as the root
    
    state = root_state.clone()
    for i in tqdm(range(num_simulations)):
        print(state.board)
        print("in simulation {}".format(i))
       
        while node["children"] != []:
            node, probas = select(node, is_stochastic)
            _transition(state, node)
            
#             placeholder fr now
        loss_threshold = 0
            
        if depth < state.max_depth-1:
            value = expand(state, node)
            results = db.select("""SELECT loss FROM data WHERE depth={}""".format(node["depth"]))
            loss_threshold = np.mean(results)*1.2
            result = value
            if value > loss_threshold:
                result = -1
        else:
            result, rowid = state.evaluate(node)
        
        board_string = convert_to_string(state.board)
        probas_string = convert_to_string(probas)
        db.update("""UPDATE data SET board_string = {}, probas_string = {}, result = {}"""
                  .format(board_string, probas_string, result))
        
        node = backup(node, value, root_node["depth"])

        state = arch_state()
#         else:
#             node = backup(node, value, root_node["depth"])
            
#     result = state.evaluate(node)
#     record.append((state.board, probas, result))
#     records.append(record)

        
#     tree starting from chosen node is saved, the rest is discarded

#     state = root_state.clone()

#     node, probas = select(node, is_stochastic, most_visits=True)
    
#     print("Node Action:", node["action"])
#     print("Node Depth:", node["depth"])
    
#     _transition(root_state, node)
    
    
#     might want to delete node["parent"] here, since in theory it wont matter

#     try:
#         del node["parent"]["parent"]
#     except:
#         pass
            
#     return node, probas, root_state, records
    return node, records

Using TensorFlow backend.


In [5]:
import database as db
db.create_tables()

In [6]:
node, records = MCTS(None, records=[], num_simulations=15, is_stochastic=True)

  0%|          | 0/15 [00:00<?, ?it/s]/home/jprothero/.conda/envs/main/lib/python3.6/site-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
in simulation 0
in expand


  7%|▋         | 1/15 [00:12<02:57, 12.66s/it]

in create node
in create node
in create node
in create node
in create node
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
in simulation 1
in select
U: 0.1409074068069458, child visits: 0, child proba: 0.1409074068069458, child mean value: 0
U: 0.2052573263645172, child visits: 0, child proba: 0.2052573263645172, child mean value: 0
U: 0.1322239637374878, child visits: 0, child proba: 0.1322239637374878, child mean value: 0
U: 0.17482885718345642, child visits: 0, child proba: 0.17482885718345642, child mean value: 0
U: 0.3467824459075928, child visits: 0, child proba: 0.3467824459075928, child mean value: 0
> <ipython-input-2-969d3bac342e>(87)select()
     85 
     86     set_trace()
---> 87     test = "hi"
     88     return node, probabilities
     89 

ipdb> node
{'children': [], 'visits': 0, 'total_value': 0, 'mean_value': 0, 'probability': 0.17482886, 'parent': {'children': [{'children': [], 'visits': 0, 'total_value': 0, 'mean_value': 0, 'probability': 0.14090741, 'parent': {...}, 'action':

BdbQuit: 

In [ ]:


# def self_play(num_games=1):
#     node = None
#     records = []
#     for i in range(num_games):
#         node, records = MCTS(node, records=[], num_simulations=10, is_stochastic=True)
        
#     return records
        
# #     I think it's better to compare to the average of a certain depth

In [5]:
# self_play()

  0%|          | 0/10 [00:00<?, ?it/s]/home/jprothero/.conda/envs/main/lib/python3.6/site-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
in simulation 0
in expand


 40%|████      | 4/10 [00:12<00:18,  3.17s/it]

in create node
in create node
in create node
in create node
in create node
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
in simulation 1
in select
U: -inf, child visits: 0, child proba: 0.1105479970574379, child mean value: 0
U: -inf, child visits: 0, child proba: 0.1256144791841507, child mean value: 0
U: -inf, child visits: 0, child proba: 0.35521194338798523, child mean value: 0
U: -inf, child visits: 0, child proba: 0.19855783879756927, child mean value: 0
U: -inf, child visits: 0, child proba: 0.2100677341222763, child mean value: 0
in transition
in expand
in create node
in create node
in create node
in create node
in create node
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
in simulation 2
in select
U: -inf, child visits: 0, child proba: 0.15526661276817322, child mean value: 0
U: -inf, child visits: 0, child proba: 0.15735231339931488, child mean value: 0
U: -inf, child visits: 0, child proba: 0.3785383403301239, child mean value: 0
U: -inf, child visits: 0, child proba: 0.14535506069660187, child 

 50%|█████     | 5/10 [00:41<00:41,  8.33s/it]

...

 80%|████████  | 8/10 [01:07<00:16,  8.44s/it]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
in simulation 8
in select
U: -inf, child visits: 2, child proba: 0.1105479970574379, child mean value: 0.0
U: 0.06035185511579037, child visits: 0, child proba: 0.1256144791841507, child mean value: 0
U: 0.17066264878049905, child visits: 0, child proba: 0.35521194338798523, child mean value: 0
U: 0.09539771208737655, child visits: 0, child proba: 0.19855783879756927, child mean value: 0
U: 0.10092767598601504, child visits: 0, child proba: 0.2100677341222763, child mean value: 0
in transition
in select
U: 0.07459831206533912, child visits: 0, child proba: 0.15526661276817322, child mean value: 0
U: 0.07560039321970222, child visits: 0, child proba: 0.15735231339931488, child mean value: 0
U: 0.1818698864952019, child visits: 0, child proba: 0.3785383403301239, child mean value: 0
U: -inf, child visits: 2, child proba: 0.14535506069660187, child mean value: 0.0
U: 0.07854815945662834, child visits: 0, child proba: 0.16348770260810852, child mean valu

100%|██████████| 10/10 [01:16<00:00,  7.61s/it]

U: 0.0, child visits: 0, child proba: 0.22544965147972107, child mean value: 0
in transition
in select
U: 0.0, child visits: 0, child proba: 0.25717341899871826, child mean value: 0
U: -inf, child visits: 1, child proba: 0.24197621643543243, child mean value: -1.0
U: 0.0, child visits: 0, child proba: 0.12575818598270416, child mean value: 0
U: 0.0, child visits: 0, child proba: 0.128472700715065, child mean value: 0
U: 0.0, child visits: 0, child proba: 0.24661947786808014, child mean value: 0
in transition
in select
U: 0.0, child visits: 0, child proba: 0.15731754899024963, child mean value: 0
U: 0.0, child visits: 0, child proba: 0.2476944476366043, child mean value: 0
U: 0.0, child visits: 0, child proba: 0.18900007009506226, child mean value: 0
U: 0.0, child visits: 0, child proba: 0.21060429513454437, child mean value: 0
U: -inf, child visits: 1, child proba: 0.19538363814353943, child mean value: -1.0
in transition
in select
U: -inf, child visits: 0, child proba: 0.2327947914600

[(array([0, 3, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  array([ 0.2,  0.2,  0.2,  0.2,  0.2]),
  1),
 (array([0, 3, 1, 2, 0, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  array([ 0.2,  0.2,  0.2,  0.2,  0.2]),
  -1)]

In [6]:
# def twosFloor(num):
#     if num % 2 == 1:
#         return (num - 1) // 2
#     else:
#         return num // 2

# def shiftedTwosFloor(num):
#     num += 3
#     if num % 2 == 1:
#         return (num - 1) // 2
#     else:
#         return num // 2

# new_rep = [3, 3, 0, 4, 2, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
#            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,]

# new_rep = [np.random.randint(0, 4) for _ in range(19)]

# max_depth=10
# max_width=6
# size = len(new_rep)


# M = []
# for _ in range(np.ceil(size/2)):
#     M.append([])
#     for _ in range(np.ceil(size/2)):
#         M[-1].append(0)
# skip_num = 0
# ready = False
# for i in range(size):
#     if skip_num % (max_width-1)
#     i = i % (max_depth-1)
#     if i == 0:
        
        
#     if (max_width-1-skip_num) > 0 and i % (max_width-1-skip_num) == 0 and ready:
#         skip_num += 1
#     if (max_width-skip_num-1) > 0:
#         i = i % (max_width-skip_num-1)
#         ready = True
#         i_p = skip_num
#         j_p = i+1
    
# #     0 = [0, 1]
# #     0 = [0, 2]
# #     0 = ...
# #     max_width-1 = [0, max_width]
#     try:
# #         for _ in range(skip_num):
# #             i_p = twosFloor(i_p)
# #             j_p = shiftedTwosFloor(i_p)
#         M[i_p][j_p] = new_rep[i]
#     except:
#         print(i_p, j_p)

# np.array(M)

In [ ]:
# what would you pass into a single alphazero net
# 

In [ ]:
# idea: instead of per layer scoring, could do final model scoring.
# since there is a no layer options, all configurations are reachable

# issues: less data points
# benefits: skip connections don't make a lot of sense if they arent connected to another layer


# potential halfway solution: have the scoring only happen on non skip connections, and at the end
# I like how that sounds

# expanding on that: I think that excluding the evaluation for one node also makes sense,
# maybe I can shift the depth by one 

In [ ]:
# idea: pass the skip connections as separate planes and process them through a conv
# that should remove the notion of sequentialness (which for skip connections dont matter)
# the non skip connections are passed as one hots directly to the lstm (and maybe conv too)

In [ ]:
# based on the paper, the MCTS might only be used to train alpha go zero, rather than actually be used for play

In [ ]:
# idea: realtime MCTS that will attempt to optimize something while running (for example as a keras optimizer
# callback)

In [ ]:
# idea, rather than having nodes, have a state cache which will reduce redunancy
# when a same state is reached, have it point to that existing state